In [49]:
import numpy as np
import matplotlib.pyplot as plt
import random

import warnings

In [50]:
# GEOMETRICAL PARAMETERS
b = 10e-2

# NUMERICAL PARAMETERS
dt = 50e-3

alpha = 1e-2

threshold = 11 # Mahalanobis distance threshold for outlier rejection

# Kalman filter for pose estimation

We represent the pose of the robot with a state vector $X = [x,y,\theta]^T$, where $x$ and $y$ are the coordinates of the robot in the plane of motion and $\theta$ is the heading of the robot w.r.t the horizontal ($x$-axis).

The control of the robot is achieved through setting the speeds of the right and left motors of the robot. These variables are accessed as ```motor.speed.right``` and ```motor.speed.left``` respectively and are given as an integer  in the range $[\![-500,500]\!]$ where $-500$ corresponds to a speed of $-20 cm.s^{-1}$

We deal with discretized timesteps. At a given timestep $i$:

*   State vector $X_i = [x_i,y_i,\theta_i]^T$
*   Control vector $U_i = [v_i^R,v_i^L]^T$

Kalman filtering relies on 2 steps:

1.   The **prediction step** that uses propriocetive sensors (in this case, the encoders giving left and right motor speeds) to get a "blind" estimation of the robot's pose.
2.   The **correction step** that uses external sensors (in this case, the camera's image) to have a measurement of the robot's pose.

The result of the Kalman filtering is a pose estimation of the Thymio that is a weighted average of the information gathered in the prediction and correction steps. These weights reflect the confidence that we have in the different measurements.

## Prediction step

We start by defining a dynamical model of our system:

$$\forall i \in \mathbb{N}, \;\; X_{i+1} = f(X_i,U_i) = \begin{bmatrix}x_i + \frac{v_i^R+v_i^L}{2}\cos(\theta)\cdot \Delta t \\ y_i + \frac{v_i^R+v_i^L}{2}\sin(\theta)\cdot \Delta t \\ \theta_i + \frac{v_i^R-v_i^L}{b}\cdot\Delta t\end{bmatrix} $$

We can then define the following matrices:


*   The **Jacobian w.r.t state**
$$ J_i = \begin{bmatrix} \frac{\partial f}{\partial X} \end{bmatrix}_i = \begin{bmatrix}1 &0 & -\frac{v_i^R+v_i^L}{2}\sin(\theta_i)\cdot \Delta t \\0 & 1 & \frac{v_i^R+v_i^L}{2}\cos(\theta_i)\cdot \Delta t  \\ 0 & 0 & 1\end{bmatrix}$$
*   The **control-to-state Jacobian**
$$ G_i = \begin{bmatrix} \frac{\partial f}{\partial U} \end{bmatrix}_i = \begin{bmatrix} \frac{1}{2} \cos(\theta_i) \cdot \Delta t & \frac{1}{2} \cos(\theta_i) \cdot \Delta t \\ \frac{1}{2} \sin(\theta_i) \cdot \Delta t & \frac{1}{2} \sin(\theta_i) \cdot \Delta t\\ \frac{1}{b} \cdot \Delta t & -\frac{1}{b} \cdot \Delta t\end{bmatrix}$$


In the prediction step, we then use the dynamical model of the system to compute an estimated pose of the robot at timestep $i+1$, assuming the pose and control are known at timestep $i$:

$$ \hat{X}_{i+1|i} = f(\hat{X}_{i|i},U_{i|i}) $$

Moreover, we use the Jacobian matrices defined above to compute the uncertainty matrix associated to this estimation. This will later allow us to weigh the relative importance of this estimated pose, as compared to the measured pose we will infer from the camera:

$$ P_{i+1|i} = J_i P_{i|i} J_i^T + Q_i, \; \; Q_i = G_i \begin{bmatrix} \sigma^2_{v^R_i} & 0 \\\ 0 & \sigma^2_{v^L_i} \end{bmatrix} G_i^T + \alpha \; \mathbb{I}_{3\times3}$$

Where $\alpha > 0$ and $\sigma^2_{v^R_i}$ and $\sigma^2_{v^L_i}$ are the standard deviations associated with the measures of the speeds of the right and left motors respectively.

These variance terms can be measured by running the motors at a constant given speed and looking at the distribution of the measurements obtained over time. We can then check the Gaussianity of said distribution and obtain the coefficients by computing their standard deviations.

In [51]:
## CODE FOR CALIBRATING THE MOTOR SPEED STANDARD DEVIATIONS
# Test sequence:
# run right motor for 5 seconds at a set speed and measure the reported values. Plot a histogram of the values obtained (verify gaussianity), compute mean (verify mean = set speed value) and variance (get var_v_r).
# repeat sequence for the left motor

In [52]:
## CODE FOR UPDATING X, J, G, P
# Inputs : X_i|i, U_i
# Outputs : X_i+1|i, P_i+1|i

def convert_speed_to_tymio_int(v_speed):
  if np.abs(v_speed) > 20e-2:
    warnings.warn('Demanded speed is outside Thymio range and has been capped')
    return np.sign(v_speed) * 500
  else:
    return np.floor(v_speed * 500 / (20e-2))

def convert_thymio_int_to_speed(motor_int):
  return motor_int * 20e-2 / 500

def motion_model(X,U,dt):
  # Returns the next pose of the Tymio robot predicted by the motion model for an inital pose X = [x[m],y[m],theta[rad]], a control U = [v_r[m/s],v_l[m/s]] and a time step dt
  # Inputs: X (3x1 numpy array), U (2x1 numpy array), dt (positive float)
  # Outputs : X+ (3x1 numpy array)

  x = X[0]
  y = X[1]
  theta = X[2]
  v_r = U[0]
  v_l = U[1]

  x_plus = x + (v_r+v_l)/2 * np.cos(theta) * dt
  y_plus = y + (v_r+v_l)/2 * np.sin(theta) * dt
  theta_plus = theta + (v_r-v_l)/b * dt

  return np.array([x_plus,y_plus,theta_plus]).T

def estimate_next_pose(X_meas,P,U_sens,dt):
  # Returns the next pose of the Tymio robot predicted by the motion model for a measured pose X_meas = [x[m],y[m],theta[rad]], a sensor value for the control input U_sens = [v_r[int],v_l[int]] and a time step dt
  # as well as the uncertainty associated with this estimation
  # Inputs: X_meas (3x1 numpy array), P (3x3 numpy array), U_sens (2x1 numpy array), dt (positive float)
  # Outputs : X_estim (3x1 numpy array), P_estim (3x3 numpy array)

  x = X_meas[0]
  y = X_meas[1]
  theta = X_meas[2]
  v_r = convert_thymio_int_to_speed(U_sens[0])
  v_l = convert_thymio_int_to_speed(U_sens[1])

  X_estim = motion_model(X_meas,np.array([v_r,v_l]).T,dt)

  J = np.array([[1,0,-(v_r+v_l)/2 * np.sin(theta) * dt],[0,1,(v_r+v_l)/2 * np.cos(theta) * dt],[0,0,1]])
  G = np.array([[0.5*np.cos(theta),0.52*np.cos(theta)],[0.5*np.sin(theta),0.5*np.sin(theta)],[dt/b, -dt/b]])

  Q = G @ np.array([[var_v_r,0],[0,var_v_l]]) @ G.T + alpha * np.eye(3)

  P_estim = J @ P @ J.T + Q

  return X_estim, P_estim

## Correction step

For the correction step, we must define a measurement model that allows us to map from the measured variable $z$ to the state vector $X$. In the case of the camera we have the following:

$$\forall i \in \mathbb{N}, \;\; Z_{i} = h(X_i) = \begin{bmatrix}x_i \\ y_i \\ \theta_i \end{bmatrix}$$

We can then define the **measurement Jacobian** as:

$$ H = \begin{bmatrix} \frac{\partial h}{\partial X} \end{bmatrix} = \mathbb{I}_{3\times3}$$

From the measurement, we can compute the **residual** - or innovation - $Y_i$ to the predicted pose as

$$ Y_i = Z_i - H \: \hat{X}_{i+1|i} = Z_i - \hat{X}_{i+1|i}$$

We also compute the predicted covariance of this residual as:
$$ S_i = H \: P_{i+1|i} \: H^T + R = P_{i+1|i} + R, \; \; \; \; R = \begin{bmatrix} \sigma^2_{\theta,cam} & 0 \ 0 \\\ 0 & \sigma_{y,cam}^2 & 0 \\\ 0 & 0 & \sigma_{\theta,cam}^2 \end{bmatrix} $$

Similarly to previously, $R$ contains the values of standard deviations associated with the camera measurements of the pose. These coefficients can be estimated by measuring the distribution of the the pose measurements for a fixed state. The coefficients can then be replaced by the numerical values of the standard deviations of these measurement distributions.


Moreover, this covariance can be used as a way to detect outliers in measured values. Indeed, if the camera loses track of the robot or measures an improbable pose of the robot considering its previous pose, we muste exclude this measurement as it can throw the pose estimation off.
To detect outliers, we can use the Mahalanobis distance for $Y_i$ defined as:
$$ d_M^2 = Y_i^T \: S_i^{-1} \: Y_i$$
Assuming the residuals $Y_i$ follow a Gaussian distribution of covariance $S$, then $d_M^2$ follows a $\chi^2$ distribution with 3 degrees of freedom (dimension of measurement vector). This distance then gives a sort of measure of the residual in units of variance, and thus a high value of $d_M^2$ indicates that the measurement is very improbable and most likely is an outlier.

For a 3-dimensional measurement space, we can reject the measurement with 99% certainty if this distance value is above 11,34.

The next step consists in computing the weight we give to the predicted pose and the measured pose. Intuitively, it can be seen as the proportion of total uncertainty in the pose estimation that comes from the measurement. Mathematically, this matrix is given by:

$$ K = P_{i+1|i} \: H^T \: S_i^{-1} = P_{i+1|i} \: S_i^{-1} = P_{i+1|i} \: (P_{i+1|i} + R)^{-1} $$

Finaly, the last step is to update the pose estimation and the uncertainty matrix associated with this estimation, given the extra information given by the camera:

$$ \hat{X}_{i+1|i+1} = \hat{X}_{i+1|i} + K \: Y_i $$
$$ P_{i+1|i+1} = (\mathbb{I}_{3\times3} - KH) \: P_{i+1|i} = (\mathbb{I}_{3\times3}-K)\:P_{i+1|i} $$

In [53]:
## CODE FOR CALIBRATING THE CAMERA STANDARD DEVIATIONS
# Test sequence :
# place the tymio in a given pose and run the video tracking for at least 200 frames. Plot a histogram of each of the residual's coordinates (verify gaussianity) (/!\ wrap the angle residual to ]-pi,pi]). Compute mean (verify that it corresponds to ground truth) and variance for each measured quantity (to get the coefficients of R)
# optional: repeat for different poses and keep the highest values for each variance across the different poses

In [54]:
## CODE FOR COMPUTING Y, S
# Input : X_i+1|i, P_i+1|i, Z_i (camera data)
# Output : Y_i, S_i^-1

In [55]:
## CODE FOR OUTLIER REJECTION
# Check the Mahalanobis distance
# If the measurement is an outlier, set K to 0 (-> disregard the measurement in the next step) <=> skip the next step and loop back to prediction

In [56]:
## CODE FOR UPDATING K, X, P
# Inputs: X_i+1|i, P_i+1|i, Y_i, S_i^-1
# Outputs: X_i+1|i+1, P_i+1|i+1

In [57]:
def correct_estimation(X_estim,P_estim,Z_meas):
  # Returns the corrected pose of the Tymio robot (based on the EKF) based on the estimated pose (given by the control model) X_estim = [x[m],y[m],theta[rad]], and the measured pose Z_meas = [x_cam[m],y_cam[m],theta_cam[rad]]
  # as well as the uncertainty associated with this corrected position
  # Inputs: X_estim (3x1 numpy array), P_estim (3x3 numpy array), Z_meas (3x1 numpy array)
  # Outputs : X_corr (3x1 numpy array), P_corr (3x3 numpy array)

  Y = Z_meas - X_estim # compute the residual
  Y[2] = (Y[2] + np.pi) % (2*np.pi) - np.pi # wrap the residual angle to ]-pi,pi]

  S = P_estim + np.array([[var_x_cam,0,0],[0,var_y_cam,0],[0,0,var_theta_cam]])
  S_inv = np.linalg.inv(S)

  # compute Mahalanobis distance and abort correction step if the measurement is rejected
  if Y.T @ S_inv @ Y > threshold:
    warnings.warn('Measurement rejected')
    return X_estim, P_estim

  K = P_estim @ S_inv

  X_corr = X_estim + K @ Y
  P_corr = (np.eye(3) - K) @ P_estim

  return X_corr, P_corr

In [58]:
# TEST CASE
X_0 = np.array([0,0,0]).T
P_0 = np.zeros((3,3))
U_cont = np.array([10e-2,10e-2]).T # actual speed in m/s
U_cont_int = np.array([convert_speed_to_tymio_int(U_cont[0]),convert_speed_to_tymio_int(U_cont[1])]).T # actual speed in int

var_v_l = convert_thymio_int_to_speed(np.sqrt(10))**2
var_v_r = convert_thymio_int_to_speed(np.sqrt(10))**2

U_sens = U_cont_int + np.array([np.floor(random.gauss(0,10)),np.floor(random.gauss(0,10))]).T # sensed speed with sensor noise

print("---- ESTIMATION STEP ----")
print()

print("Actual speed command :", U_cont_int)
print("Sensed speed command :", U_sens)

X_estim, P_estim = estimate_next_pose(X_0,P_0,U_sens,dt) # estimation step

print()
print("Actual pose (w/o noise) :", motion_model(X_0,U_cont,dt))
print("Estimated pose (w/ noise) :", X_estim)
print(P_estim)
print()
print()



var_x_cam = 1e-6 # 1mm error in cam
var_y_cam = 1e-6 # 1mm error in cam
var_theta_cam = 1e-2 # 0.1 rad error in cam

Z_meas = np.array([5.2e-3,1e-4,6e-3]).T # camera's measured pose
X_corr, P_corr = correct_estimation(X_estim,P_estim,Z_meas) # correction step

print("---- CORRECTION STEP ----")
print()
print("Measured position :", Z_meas)
print("Pose :",X_corr)
print(P_corr)

---- ESTIMATION STEP ----

Actual speed command : [250. 250.]
Sensed speed command : [258. 256.]

Actual pose (w/o noise) : [0.005 0.    0.   ]
Estimated pose (w/ noise) : [0.00514 0.      0.0004 ]
[[ 1.00008326e-02  0.00000000e+00 -1.60000000e-08]
 [ 0.00000000e+00  1.00000000e-02  0.00000000e+00]
 [-1.60000000e-08  0.00000000e+00  1.00008000e-02]]


---- CORRECTION STEP ----

Measured position : [0.0052 0.0001 0.006 ]
Pose : [5.19999400e-03 9.99900010e-05 3.20011195e-03]
[[ 9.99900018e-07  0.00000000e+00 -7.99821423e-13]
 [ 0.00000000e+00  9.99900010e-07  0.00000000e+00]
 [-7.99821423e-13  0.00000000e+00  5.00019999e-03]]
